In [1]:
# Алексеев Д.П. (DSU-4, NUMPY-21)
# Домашнее задание «Основы парсинга и работы с API» (#5)
# Задание 1.
# Обязательная часть
# Будем парсить страницу со свежими новостями на habr.com/ru/all/.

# Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 
# Эти слова определяем в начале кода в переменной, например:
# KEYWORDS = [‘python’, ‘парсинг’]

# Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

# В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

# Дополнительная часть (необязательная)
# Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
# Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
# Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

# Задание 2.
# Обязательная часть
# Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. 
# https://www.avast.com/hackcheck/result-leaks
# Список email-ов задаем переменной в начале кода:
# EMAIL = [xxx@x.ru, yyy@y.com]

# В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

# Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

# Дополнительная часть (необязательная)
# Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
# Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get
# GROUP = 'netology’
# TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
# В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [182]:
import requests

In [183]:
from bs4 import BeautifulSoup

In [184]:
import pandas as pd

In [204]:
# определяем список хабов, которые нам интересны
KEYWORDS = ['автоматизация', 'модели']

In [205]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [206]:
# извлекаем посты с новостями
posts = soup.find_all('article', class_= 'tm-articles-list__item')
posts

[<article class="tm-articles-list__item" data-navigatable="" id="578860" tabindex="0"><div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/alexzfort/" title="alexzfort"><div class="tm-entity-image"><img alt="" class="tm-entity-image__pic" height="24" loading="lazy" src="//habrastorage.org/r/w32/getpro/habr/avatars/d2a/e8a/5ac/d2ae8a5ac9876f838ebdd45f3ea3bc3b.png" width="24"/></div></a> <span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/alexzfort/">
       alexzfort
     </a> </span></span> <span class="tm-article-snippet__datetime-published"><time datetime="2021-09-19T20:06:24.000Z" title="2021-09-19, 23:06">вчера в 23:06</time></span></div> <!-- --></div> <h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title-link" data-article-link="" href=

In [209]:
df = pd.DataFrame(columns=['Date', 'Title', 'Link'])
pd.set_option('display.max_colwidth', None) #опция для того, чтобы в датафрейме не обрезалось содержимое при отображении
i = 0

for post in posts:
    all_articles = post.find_all(class_ = 'article-formatted-body article-formatted-body_version-1')
    for article in all_articles:
        if any(keyword in article.text for keyword in KEYWORDS): #поиск любого ключевого слова из списка в тексте статьи
            time_found = post.find(class_ = 'tm-article-snippet__datetime-published') #находим дату и время публикации
            title_found = post.find('a', class_ = 'tm-article-snippet__title-link') #находим заголовок публикации
            href_ = 'https://habr.com'+title_found.attrs.get('href') #находим ссылку на публикацию и составляем полный адрес
            df.loc[i] = [time_found.time['datetime'], title_found.text, href_] #записываем в датафрейм
            i += 1
        break
        
df    

,Date,Title,Link
0,2021-09-19T14:46:52.000Z,"Больше механических клавиатур хороших и разных: новые модели, на которые стоит обратить внимание",https://habr.com/ru/company/selectel/blog/578238/
1,2021-09-19T14:38:32.000Z,Дайджест интересных материалов для мобильного разработчика #411 (13 — 19 сентября),https://habr.com/ru/company/productivity_inside/blog/578824/


In [ ]:
# Задание 2.
# Обязательная часть
# Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. 
# https://www.avast.com/hackcheck/result-leaks
# Список email-ов задаем переменной в начале кода:
# EMAIL = [xxx@x.ru, yyy@y.com]
# В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>
# Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

In [62]:
import requests
import time

In [63]:
from bs4 import BeautifulSoup

In [74]:
# список адресов, передаваемых в скрипт для проверки
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [77]:
# из режима разработчика получаем ссылку непосредственно на сервис проверки e-mail-ов,
# т.к. адрес https://www.avast.com/hackcheck/result-leaks - это просто "витрина".
# см.раздел General, поле RequestURL
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

# в разделе Headers в режиме разработчика для "unauthorized-data" 
# берем корректное название заголовков (раздел Request Headers)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0'
}

# в разделе Headers в режиме разработчика для "unauthorized-data" берем корректное название параметров (раздел Request Payload)
PARAMS = {
    'emailAddresses': EMAIL
}

In [78]:
time.sleep(0.3)
res = requests.post(URL, headers=HEADERS, json=PARAMS)
res

<Response [200]>

In [79]:
res.text

'{"breaches":{"2":{"breachId":2,"site":"linkedin.com","recordsCount":158591429,"description":"In 2012, online professional networking platform LinkedIn fell victim to a breach of its members\' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \\n\\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":111975337,"emails":158591429}},"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe\'s corporate network and the stole source code for several of its software products. The affected products included Adobe\'s ColdFusion w

In [81]:
import json

In [90]:
import pandas as pd

In [117]:
# идентификатор утечки - "breachId"
# <дата утечки> - "publishDate"
# <источник утечки> - "site"
# <описание утечки> - "description"

# в "summary" перечислены для каждого e-mail из запроса все идентификаторы утечек (breachId)

res_json = json.loads(res.text)
# res_json
res_df = pd.json_normalize(res_json)#['breaches'])
res_df

,breaches.2.breachId,breaches.2.site,breaches.2.recordsCount,breaches.2.description,breaches.2.publishDate,breaches.2.statistics.usernames,breaches.2.statistics.passwords,breaches.2.statistics.emails,breaches.3.breachId,breaches.3.site,...,data.globalreach.eu.yyy@y.com,data.cfire.mail.ru.xxx@x.ru,data.vk.com.xxx@x.ru,data.forums.vkmonline.com.yyy@y.com,data.netlog.com.yyy@y.com,data.imesh.com.xxx@x.ru,data.imesh.com.yyy@y.com,data.zynga.com.yyy@y.com,summary.xxx@x.ru.breaches,summary.yyy@y.com.breaches
0,2,linkedin.com,158591429,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z,0,111975337,158591429,3,adobe.com,...,"[{'breachId': 3520, 'usernameBreached': False,...","[{'breachId': 3164, 'usernameBreached': True, ...","[{'breachId': 12, 'usernameBreached': False, '...","[{'breachId': 37177, 'usernameBreached': True,...","[{'breachId': 13662, 'usernameBreached': False...","[{'breachId': 15, 'usernameBreached': True, 'p...","[{'breachId': 15, 'usernameBreached': True, 'p...","[{'breachId': 17009, 'usernameBreached': True,...","[3, 12, 15, 2961, 3164, 3176]","[2, 3, 15, 41, 3520, 3587, 3669, 13094, 13662,..."
